In [6]:
# let's first write the forward functions
import numpy as np
import random

def softmax(x: np.ndarray, 
            normalization: bool = True) -> np.ndarray:
    # first squeeze the array in case it is multi-dimentional
    x = np.squeeze(x)

    if len(x.shape) != 1:
        raise ValueError(f"The input is expected to be 1 dimensional. Found: {len(x.shape)} dimensions")

    # the main idea behind 'normalization' is to avoid numerical overflow with the softmax function 
    # (mainly with the denominator as a sum of exponential functions). The output of softmax for (x1, x2, ... xn) is the same as the output for
    # (x1 - C, x2 - C, ... xn - C) where C is any constant. (other operations such as division will alter the output)
    # consider the following link for the mathematical details : https://jaykmody.com/blog/stable-softmax/

    max_x = max(x) if normalization else 0
    x = x - max_x # if normalization is False, then 'x' is the same as the original output, otherwise the maximum element will be subtracted from each element
    sum_exp = np.sum(np.exp(x))
    return np.exp(x) / sum_exp

# let's test the function 

def test_softmax(num_tests: int = 200):
    for _ in range(num_tests):
        # generate a random array
        random_x = np.random.random(size=(10,)) * 10 - random.random() 
        soft_x = softmax(random_x, normalization=False)
        soft_x_norm = softmax(random_x)

        assert np.allclose(soft_x, soft_x_norm), "WELL recheck the code!!" # the results are less than 10 ** -5 apart.
    print("well the normalization works")

In [7]:
test_softmax()

well the normalization works


let's find the gradient of the sofmax function using backpropagation: 
$$ 
softmax(x_1, x_2, ... x_n) = \begin{bmatrix}
\frac{e^{x_1}}{S} \\
\frac{e^{x_2}}{S} \\
..  \\
\frac{e^{x_n}}{S}
\end{bmatrix}
$$
where $S = \sum_{i = 1}^n e^{x_i}$. Let's denote $S_i$ the output of the softmax, Then the gradient of softmax function can be expressed as follows:
$$
\begin{bmatrix}
\frac{d S1}{d x_1} &\frac{d S1}{d x_2} & ..  &\frac{d S1}{d x_n} \\
\frac{d S2}{d x_1} &\frac{d S2}{d x_2} & ..  &\frac{d S2}{d x_n} \\ 
..  \\
\frac{d Sn}{d x_1} &\frac{d Sn}{d x_2} & ..  &\frac{d Sn}{d x_n} \\ 
\end{bmatrix}
$$
let's find $\frac{d S_i}{d xi}$ and $\frac{d S_i}{d xj}$
if we define $f(x) = \frac{x}{x + k}$ where $k$ is some constant and $g = e^{x}$, then 
$$S_i = f(g(x_i))$$ 

\begin{align}
\frac{d S_i}{d xi} = \frac{d f}{d g} \cdot \frac{d g }{dxi} 
\end{align}
Using
1. $f(x) = \frac{x}{x + k}$ and $\frac{d f}{d x} = -\frac{k}{(x + k) ^ 2}$ 
2. $g = e^{x}$ and $\frac{dg}{dx} = g(x)$
then 
$$ 
\frac{d S_i}{d xi} = -\frac{k}{(g + k) ^ 2} \cdot g
$$
where $g = e^{x_i}$ and $k = \sum_{l \neq i}^n e^{x_i}$ (which is constant with respect to $x_i$). The final result will be:

$$ 
\frac{d S_i}{d x_i} = \frac{e^{x_i} \cdot (\sum_{l \neq i}^n e^{x_i})}{(\sum_{l = 1}^n e^{x_l}) ^ 2} = S_i - S_i ^ 2
$$
Using the same process we can derive the other derivative: $$, 
$$ 
\frac{d S_i}{d xj} = -S_i \cdot S_j
$$

In [16]:
# let's implement this function
def calculate_gradient(x: np.ndarray, normalization: bool = True) -> np.ndarray:
    # the first step is to calculate the output of softmax on x
    s = softmax(x, normalization=normalization)
    s = np.expand_dims(s, axis=-1)
    assert s.shape == (len(x), 1)
    gradient =  - s @ np.transpose(s)
    # the current variable contains only - S_i . S_j for all i, j. the S_i term should be added to the diagonal entries
    for i in range(len(x)):
        gradient[i][i] += s[i][0]
    return gradient

In [17]:
calculate_gradient(x=[1,2, 3])

array([[ 0.08192507, -0.02203304, -0.05989202],
       [-0.02203304,  0.18483645, -0.1628034 ],
       [-0.05989202, -0.1628034 ,  0.22269543]])